In [24]:
import os
import sys
import glob
import pickle
import numpy as np
import pandas as pd
import catboost
import seaborn as sns
from pathlib import Path
import matplotlib.pyplot as plt
from itertools import combinations
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics.cluster import adjusted_rand_score

In [9]:
df_coreMLST = pd.read_csv(
    "../../Data/10359_coreMLST_dataframe.txt",sep='\t')

<ipython-input-9-6d77e7070be9>:1: DtypeWarning: Columns (4,158,182,204,341,376,454,671,728,749,750,782,871,901,948,1111,1163,1289) have mixed types. Specify dtype option on import or set low_memory=False.
  df_coreMLST = pd.read_csv(


In [10]:
df_coreMLST.head()

,id,CAMP0001,CAMP0002,CAMP0003,CAMP0006,CAMP0007,CAMP0009,CAMP0010,CAMP0012,CAMP0013,...,CAMP1846,CAMP1847,CAMP1848,CAMP1849,CAMP1850,CAMP1851,CAMP1852,CAMP1853,CAMP1854,cgST (C. jejuni / C. coli cgMLST v1.0)
0,2659,1,37.0,40.0,14.0,174.0,25.0,1.0,30.0,1.0,...,1,1.0,1,1,1,1,17,2,1,26
1,2663,152,27.0,666.0,71.0,177.0,36.0,28.0,31.0,351.0,...,1,1.0,1,1,1,2,17,5,2,28
2,2670,142,94.0,111.0,14.0,117.0,48.0,35.0,2.0,1.0,...,1,1.0,1,1,1,1,1,1,1,32
3,2672,31,58.0,61.0,35.0,67.0,37.0,29.0,42.0,37.0,...,1,1.0,1,1,3,1,17,2,2,34
4,5858,1,30.0,29.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1,1.0,1,1,1,1,17,1,2,58


In [19]:
df_MLST_Meta = pd.read_excel(
    "../../Data/10359_Dataframe_metadata.xlsx")

In [20]:
df_MLST_Meta.head()

,id,isolate,aliases,country,continent,region,year,month,isolation_date,received_date,...,datestamp,aspA,glnA,gltA,glyA,pgm,tkt,uncA,ST (MLST),clonal_complex (MLST)
0,2659,H133940557,NaN,UK,Europe,NaN,2013.0,NaN,NaN,NaN,...,2019-12-25,2,4,1,4,19,1,5,918,ST-48 complex
1,2663,H134660450,NaN,UK [England],Europe,West Midlands,2013.0,11.0,2013-11-14,NaN,...,2019-11-15,8,17,5,2,10,3,1,524,ST-353 complex
2,2670,H140620012,NaN,UK [England],Europe,"Bedfordshire, Hertfordshire and Northamptonshire",2014.0,2.0,2014-02-04,NaN,...,2020-12-25,2,1,12,3,2,1,5,50,ST-21 complex
3,2672,H140940808,NaN,UK [England],Europe,Cumbria and Lancashire,2014.0,2.0,2014-02-26,NaN,...,2020-10-25,8,10,2,2,11,12,6,354,ST-354 complex
4,5858,OXC1,F73024; Hn_1; WTCHG_556540_222168,UK [England],Europe,Oxfordshire,2003.0,9.0,NaN,NaN,...,2020-01-29,2,1,1,3,2,1,5,21,ST-21 complex


In [23]:
df_combined = pd.merge(df_coreMLST, df_MLST_Meta, left_on='id', right_on = 'id', how='outer')

In [26]:
path2model = '../../Data/aspA-glnA-gltA-glyA-pgm-tkt-uncA-trained.pkl'
with open(path2model, 'rb') as fh:
    model = pickle.load(fh)
    

In [56]:
loci = ['glnA','gltA','glyA','pgm','tkt','uncA', 'aspA']
model.predict(df_combined[loci])
df_combined['NG'] = model.predict(df_combined[loci])

In [58]:
df_combined.to_excel('../../Data/10359_coreMLST_metadata_MLST_combined.xlsx')

In [60]:
coremlst = [x for x in df_combined.columns if x.startswith('CAMP')]

In [65]:
df_combined[coremlst].astype(str)


,CAMP0001,CAMP0002,CAMP0003,CAMP0006,CAMP0007,CAMP0009,CAMP0010,CAMP0012,CAMP0013,CAMP0015,...,CAMP1845,CAMP1846,CAMP1847,CAMP1848,CAMP1849,CAMP1850,CAMP1851,CAMP1852,CAMP1853,CAMP1854
0,1,37.0,40.0,14.0,174.0,25.0,1.0,30.0,1.0,1.0,...,1,1,1.0,1,1,1,1,17,2,1
1,152,27.0,666.0,71.0,177.0,36.0,28.0,31.0,351.0,1.0,...,2,1,1.0,1,1,1,2,17,5,2
2,142,94.0,111.0,14.0,117.0,48.0,35.0,2.0,1.0,1.0,...,1,1,1.0,1,1,1,1,1,1,1
3,31,58.0,61.0,35.0,67.0,37.0,29.0,42.0,37.0,33.0,...,1,1,1.0,1,1,3,1,17,2,2
4,1,30.0,29.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1,1,1.0,1,1,1,1,17,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10354,242,1.0,1.0,1,297.0,1.0,1.0,1.0,1.0,1.0,...,2,1,1.0,1,1,1,1,1,1,1
10355,1,31.0,33.0,79,1993.0,1.0,1.0,1.0,84.0,1.0,...,1,1,1.0,1,1,1,1,1,2,1
10356,25,1.0,62.0,1,29.0,27.0,24.0,605.0,26.0,25.0,...,1,1,1.0,1,1,1,1,1,2,1
10357,1660,1409.0,2182.0,989,2290.0,172.0,20.0,27.0,28.0,111.0,...,2,12,2.0,1,1,1,1,10,13,2
